In [3]:
!pip3 install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 15.4 MB/s 


In [4]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
import json
import psycopg2

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [5]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

##Log 에서 사용자 조회기록 가져오기

In [6]:
db = load_db()
cursor = db.cursor()

In [7]:
load_like_log_sql = f'SELECT member_id, place_id FROM place_like'
load_click_log_sql = f'SELECT member_id, place_id FROM places_log '  

In [8]:
cursor.execute(load_click_log_sql)
result = cursor.fetchall()

In [9]:
result

[(154, 65),
 (154, 66),
 (154, 67),
 (154, 68),
 (154, 69),
 (1147, 69),
 (1147, 68),
 (1147, 67),
 (1546, 67),
 (2662, 50),
 (2662, 55),
 (184, 260),
 (2155, 891),
 (2155, 62),
 (184, 629),
 (184, 959),
 (184, 699),
 (184, 386),
 (184, 210),
 (184, 664),
 (184, 360),
 (184, 696),
 (184, 238),
 (184, 9),
 (184, 4),
 (184, 788),
 (184, 79),
 (184, 165),
 (184, 15),
 (184, 777),
 (184, 98),
 (184, 94),
 (184, 32),
 (184, 395),
 (184, 104),
 (184, 943),
 (184, 844),
 (184, 562),
 (184, 794),
 (184, 29),
 (2155, 1),
 (184, 720),
 (184, 1),
 (2155, 3),
 (184, 259),
 (184, 498),
 (184, 465),
 (184, 378),
 (184, 418),
 (184, 787),
 (184, 657),
 (184, 456),
 (184, 257),
 (184, 680),
 (184, 717),
 (184, 906),
 (184, 512),
 (184, 236),
 (184, 204),
 (184, 619),
 (184, 294),
 (184, 519),
 (184, 121),
 (184, 10),
 (184, 8)]

In [10]:
tour_data = pd.DataFrame(data=result,columns=['member','place'])

###조회 횟수 및 좋아요등은 count를 반영해서 group by member,place 해야함

In [11]:
tour_data.head(3)

,member,place
0,154,65
1,154,66
2,154,67


###추천한 아이템을 설명하기 위한 설명 데이터 불러오기

In [12]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_attraction.csv",index_col=0)
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...


##유니크한 고객ID / 유니크한 아이템 ID 를 각각 행렬의 행과 열로 만들기

In [13]:
cleaned_tour = tour_data.copy()

In [14]:
cleaned_tour['MemberID'] = tour_data['member'].astype(int)

In [ ]:
cleaned_tour